In [1]:
import numpy as np
import pandas as pd

from datetime import date
import time

import random
from random import seed
from random import random

import os, os.path
import shutil

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

import matplotlib
import matplotlib.pyplot as plt
from pylab import imshow

import h5py
import sys

In [2]:
# sys.path.append('/Users/hn/Documents/00_GitHub/Ag/NASA/Python_codes/')
sys.path.append("C:\\Users\\kirti\\Documents\\NASA_Codes\\Python_codes\\")
import NASA_core as nc
# import NASA_plot_core.py as rcp

# Metadata

In [3]:
meta_dir = "C:\\Users\\kirti\\Documents\\NASA_data\\parameters\\"
meta = pd.read_csv(meta_dir+"evaluation_set.csv")
meta_moreThan10Acr=meta[meta.ExctAcr>10]
print (meta.shape)
print (meta_moreThan10Acr.shape)
meta.head(2)

(6340, 8)
(3539, 8)


,ID,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,100010_WSDA_SF_2017,alfalfa hay,center pivot,wsda,34,34.310305,2017/09/12,Grant
1,100204_WSDA_SF_2017,alfalfa hay,center pivot,wsda,62,61.826535,2017/08/09,Grant


In [4]:
# print (len(meta.ID.unique()))
# meta_lessThan10Acr=meta[meta.ExctAcr<10]
# print (meta_lessThan10Acr.shape)

# Read Training Set Labels

In [5]:
# training_set_dir = "/Users/hn/Documents/01_research_data/NASA/ML_data/"
training_set_dir = "C:\\Users\\kirti\\Documents\\NASA_data\\ML_data\\"
ground_truth_labels = pd.read_csv(training_set_dir+"train_labels.csv")
print ("Unique Votes: ", ground_truth_labels.Vote.unique())
print (len(ground_truth_labels.ID.unique()))
ground_truth_labels.head(2)

Unique Votes:  [2 1]
1849


,ID,Vote
0,99837_WSDA_SF_2017,2
1,114615_WSDA_SF_2017,1


### Detect how many fields are less than 10 acres and report in the paper

In [6]:
print (len(meta[meta.ID.isin(list(ground_truth_labels.ID))].ID.unique()))
meta.head(2)

1849


,ID,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,100010_WSDA_SF_2017,alfalfa hay,center pivot,wsda,34,34.310305,2017/09/12,Grant
1,100204_WSDA_SF_2017,alfalfa hay,center pivot,wsda,62,61.826535,2017/08/09,Grant


# Read the data

In [7]:
VI_idx = "EVI"
data_dir = "C:\\Users\\kirti\\Documents\\NASA_data\\VI_TS\\04_regularized_TS\\"

In [8]:
file_names = ["regular_Walla2015_EVI_JFD.csv", "regular_AdamBenton2016_EVI_JFD.csv", 
              "regular_Grant2017_EVI_JFD.csv", "regular_FranklinYakima2018_EVI_JFD.csv"]

data=pd.DataFrame()

for file in file_names:
    curr_file=pd.read_csv(data_dir + file)
    curr_file['human_system_start_time'] = pd.to_datetime(curr_file['human_system_start_time'])
    
    # These data are for 3 years. The middle one is the correct one
    all_years = sorted(curr_file.human_system_start_time.dt.year.unique())
    if len(all_years)==3 or len(all_years)==2:
        proper_year = all_years[1]
    elif len(all_years)==1:
        proper_year = all_years[0]

    curr_file = curr_file[curr_file.human_system_start_time.dt.year==proper_year]
    data=pd.concat([data, curr_file])

data.reset_index(drop=True, inplace=True)
data.head(2)

,ID,human_system_start_time,EVI
0,135073_WSDA_SF_2015,2015-01-10,0.049624
1,135073_WSDA_SF_2015,2015-01-20,0.026503


In [9]:
ground_truth = data[data.ID.isin(list(ground_truth_labels.ID.unique()))].copy()
len(ground_truth.ID.unique())

1849

# Toss Smalls

In [10]:
ground_truth_labels_extended = pd.merge(ground_truth_labels, meta, on=['ID'], how='left')
ground_truth_labels = ground_truth_labels_extended[ground_truth_labels_extended.ExctAcr>=10].copy()
ground_truth_labels.reset_index(drop=True, inplace=True)

print ("There are [{:.0f}] fields in total whose area adds up to [{:.2f}].".format(len(ground_truth_labels_extended), \
                                                                     ground_truth_labels_extended.ExctAcr.sum()))


print ("There are [{:.0f}] fields larger than 10 acres whose area adds up to [{:.2f}].".format(len(ground_truth_labels), \
                                                                    ground_truth_labels.ExctAcr.sum()))


There are [1849] fields in total whose area adds up to [85573.68].
There are [1342] fields larger than 10 acres whose area adds up to [83430.79].


In [11]:
ground_truth = ground_truth[ground_truth.ID.isin((list(meta_moreThan10Acr.ID)))].copy()
ground_truth_labels = ground_truth_labels[ground_truth_labels.ID.isin((list(meta_moreThan10Acr.ID)))].copy()

ground_truth.reset_index(drop=True, inplace=True)
ground_truth_labels.reset_index(drop=True, inplace=True)

# Sort

In [12]:
ground_truth.sort_values(by=["ID", 'human_system_start_time'], inplace=True)
ground_truth_labels.sort_values(by=["ID"], inplace=True)

ground_truth.reset_index(drop=True, inplace=True)
ground_truth_labels.reset_index(drop=True, inplace=True)

assert (len(ground_truth.ID.unique()) == len(ground_truth_labels.ID.unique()))

print (list(ground_truth.ID)[0])
print (list(ground_truth_labels.ID)[0])
print ("____________________________________")
print (list(ground_truth.ID)[-1])
print (list(ground_truth_labels.ID)[-1])
print ("____________________________________")
print (list(ground_truth.ID.unique())==list(ground_truth_labels.ID.unique()))

100048_WSDA_SF_2017
100048_WSDA_SF_2017
____________________________________
99909_WSDA_SF_2017
99909_WSDA_SF_2017
____________________________________
True


# Widen

In [13]:
EVI_colnames = [VI_idx + "_" + str(ii) for ii in range(1, 37) ]
columnNames = ["ID"] + EVI_colnames
ground_truth_wide = pd.DataFrame(columns=columnNames, 
                                index=range(len(ground_truth.ID.unique())))
ground_truth_wide["ID"] = ground_truth.ID.unique()

for an_ID in ground_truth.ID.unique():
    curr_df = ground_truth[ground_truth.ID==an_ID]
    
    ground_truth_wide_indx = ground_truth_wide[ground_truth_wide.ID==an_ID].index
    ground_truth_wide.loc[ground_truth_wide_indx, "EVI_1":"EVI_36"] = curr_df.EVI.values[:36]

In [14]:
print (len(ground_truth_wide.ID.unique()))
ground_truth_wide.head(2)

1342


,ID,EVI_1,EVI_2,EVI_3,EVI_4,EVI_5,EVI_6,EVI_7,EVI_8,EVI_9,...,EVI_27,EVI_28,EVI_29,EVI_30,EVI_31,EVI_32,EVI_33,EVI_34,EVI_35,EVI_36
0,100048_WSDA_SF_2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.072227,0.088491,...,0.111507,0.119534,0.13248,0.061173,0.055843,0.050513,0.045183,0.039854,0.045165,0.050476
1,100081_WSDA_SF_2017,0.014832,0.167038,0.319244,0.47145,0.378153,0.284855,0.198468,0.290683,0.398445,...,0.096717,0.117915,0.159058,0.121182,0.060591,0.0,0.050004,0.100009,0.102269,0.104529


# Split Train and Test Set

#### Make sure rows of ```ground_truth_allBands``` and ```ground_truth_labels``` are in the same order

In [15]:
print (ground_truth_labels.CropTyp.unique())
ground_truth_labels.head(2)

['bean, green' 'wheat' 'onion' 'pea, green' 'corn, field' 'corn, sweet'
 'bean, dry' 'yellow mustard' 'potato' 'canola' 'mint' 'grass seed'
 'carrot' 'buckwheat' 'bluegrass seed' 'grass hay' 'corn seed' 'pea, dry'
 'pea seed' 'barley hay' 'market crops' 'triticale' 'carrot seed'
 'wheat fallow' 'barley' 'alfalfa seed' 'oat hay' 'triticale hay']


,ID,Vote,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,100048_WSDA_SF_2017,1,"bean, green",rill,wsda,18,18.033240,2017/05/14,Grant
1,100081_WSDA_SF_2017,1,wheat,rill,wsda,16,15.959744,2017/08/09,Grant


In [16]:
ground_truth.head(2)

,ID,human_system_start_time,EVI
0,100048_WSDA_SF_2017,2017-01-06,0.0
1,100048_WSDA_SF_2017,2017-01-16,0.0


In [17]:
ground_truth_labels = ground_truth_labels.set_index('ID')
ground_truth_labels = ground_truth_labels.reindex(index=ground_truth_wide['ID'])
ground_truth_labels = ground_truth_labels.reset_index()

print (ground_truth_labels.ExctAcr.min())
ground_truth_labels.head(2)

10.0708703567


,ID,Vote,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,100048_WSDA_SF_2017,1,"bean, green",rill,wsda,18,18.033240,2017/05/14,Grant
1,100081_WSDA_SF_2017,1,wheat,rill,wsda,16,15.959744,2017/08/09,Grant


In [18]:
ground_truth_labels=ground_truth_labels[["ID", "Vote"]]

In [19]:
x_train_df, x_test_df, y_train_df, y_test_df = train_test_split(ground_truth_wide, 
                                                                ground_truth_labels, 
                                                                test_size=0.2, 
                                                                random_state=0,
                                                                shuffle=True,
                                                                stratify=ground_truth_labels.Vote.values)
x_test_df.shape

(269, 37)

# Start Random Forest

# Definitions

  - **Precision** Of all instances we predict $\hat y = 1$, what fraction is actually 1.
     \begin{equation}\label{eq:precision}
        \text{Precision} = \frac{TP}{TP + FP}
     \end{equation}

  - **Recall** Of all instances that are actually $y = 1$, what fraction we predict 1.
     \begin{equation}\label{eq:recall}
         \text{Recall} = \text{TPR} = \frac{TP}{TP + FN}
     \end{equation}
     
  - **Specifity** Fraction of all negative instances that are incorrectly predicted positive.
     \begin{equation}\label{eq:specifity}
        \text{Specifity} = \text{FPR} = \frac{FP}{TN + FP}\\
     \end{equation}
     
  - **F-Score** Adjust $\beta$ for trade off between  precision and recall. For precision oriented task $\beta = 0.5$.
     \begin{equation}\label{eq:Fscore}
        F_\beta = \frac{(1+\beta^2) TP}{ (1+\beta^2) TP + \beta^2 FN + FP}
     \end{equation}



In [20]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier

In [21]:
%%time
forest_1_default = RandomForestClassifier(n_estimators=100, 
                                          criterion='gini', max_depth=None, 
                                          min_samples_split=2, min_samples_leaf=1, 
                                          min_weight_fraction_leaf=0.0,
                                          max_features='sqrt', max_leaf_nodes=None, 
                                          min_impurity_decrease=0.0, 
                                          bootstrap=True, oob_score=False, n_jobs=None, 
                                          random_state=1, verbose=0, 
                                          warm_start=False, class_weight=None, 
                                          ccp_alpha=0.0, max_samples=None)

forest_1_default.fit(x_train_df.iloc[:, 1:], y_train_df.iloc[:, 1:].values.ravel())

CPU times: total: 391 ms
Wall time: 379 ms


RandomForestClassifier(max_features='sqrt', random_state=1)

In [23]:
forest_1_default_predictions = forest_1_default.predict(x_test_df.iloc[:, 1:])
forest_1_default_y_test_df=y_test_df.copy()
forest_1_default_y_test_df["prediction"]=list(forest_1_default_predictions)
forest_1_default_y_test_df.head(2)

,ID,Vote,prediction
1221,7667_WSDA_SF_2016,1,1
1334,99748_WSDA_SF_2017,1,1


In [42]:
true_single_predicted_single=0
true_single_predicted_double=0

true_double_predicted_single=0
true_double_predicted_double=0

for index_ in forest_1_default_y_test_df.index:
    curr_vote=list(forest_1_default_y_test_df[forest_1_default_y_test_df.index==index_].Vote)[0]
    curr_predict=list(forest_1_default_y_test_df[forest_1_default_y_test_df.index==index_].prediction)[0]
    if curr_vote==curr_predict:
        if curr_vote==1: 
            true_single_predicted_single+=1
        else:
            true_double_predicted_double+=1
    else:
        if curr_vote==1:
            true_single_predicted_double+=1
        else:
            true_double_predicted_single+=1
            
forest_default_confus_tbl_test = pd.DataFrame(columns=['None', 'Predict_Single', 'Predict_Double'], 
                               index=range(2))
forest_default_confus_tbl_test.loc[0, 'None'] = 'Actual_Single'
forest_default_confus_tbl_test.loc[1, 'None'] = 'Actual_Double'
forest_default_confus_tbl_test['Predict_Single']=0
forest_default_confus_tbl_test['Predict_Double']=0

forest_default_confus_tbl_test.loc[0, "Predict_Single"]=true_single_predicted_single
forest_default_confus_tbl_test.loc[0, "Predict_Double"]=true_single_predicted_double
forest_default_confus_tbl_test.loc[1, "Predict_Single"]=true_double_predicted_single
forest_default_confus_tbl_test.loc[1, "Predict_Double"]=true_double_predicted_double
forest_default_confus_tbl_test

,None,Predict_Single,Predict_Double
0,Actual_Single,218,1
1,Actual_Double,11,39


In [73]:
FD1_y_test_df_act_1_pred_2=forest_1_default_y_test_df[forest_1_default_y_test_df.Vote==1].copy()
FD1_y_test_df_act_2_pred_1=forest_1_default_y_test_df[forest_1_default_y_test_df.Vote==2].copy()

FD1_y_test_df_act_1_pred_2=FD1_y_test_df_act_1_pred_2[FD1_y_test_df_act_1_pred_2.prediction==2].copy()
FD1_y_test_df_act_2_pred_1=FD1_y_test_df_act_2_pred_1[FD1_y_test_df_act_2_pred_1.prediction==1].copy()

FD1_y_test_df_act_2_pred_1 = pd.merge(FD1_y_test_df_act_2_pred_1, ground_truth_labels_extended, on=['ID'], how='left')
FD1_y_test_df_act_1_pred_2 = pd.merge(FD1_y_test_df_act_1_pred_2, ground_truth_labels_extended, on=['ID'], how='left')

print (FD1_y_test_df_act_2_pred_1.ExctAcr.sum())
print (FD1_y_test_df_act_1_pred_2.ExctAcr.sum())

print (FD1_y_test_df_act_2_pred_1.ExctAcr.sum()-FD1_y_test_df_act_1_pred_2.ExctAcr.sum())

845.0734264584283
17.8815032814299
827.1919231769984


In [25]:
# parameters = {'n_jobs':[4],
#               'criterion': ["gini", "entropy"], # log_loss
#               'max_depth':[1, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
#               'min_samples_split':[2, 3, 4, 5],
#               'max_features': ["sqrt", "log2", None],
#               # 'min_impurity_decreasefloat':[0, 1, 2],
#               'class_weight':['balanced', 'balanced_subsample', None],
#               'ccp_alpha':[0.0, 1, 2, 3], 
#               'max_samples':[None, 1, 2, 3, 4, 5]} # , 
# forest_classifier_grid = GridSearchCV(RandomForestClassifier(random_state=0), 
#                                       parameters, cv=5, verbose=1,
#                                       error_score='raise')

# forest_classifier_grid.fit(x_train_df.iloc[:, 1:], y_train_df.Vote.values.ravel())

In [26]:
# RandomForestClassifier.get_params().keys()

In [ ]:
(n_estimators=100, 
criterion='gini', 
max_depth=None, 
min_samples_split=2, 
min_samples_leaf=1, 
min_weight_fraction_leaf=0.0,
max_features='sqrt', 
max_leaf_nodes=None, 
min_impurity_decrease=0.0, 
bootstrap=True, 
oob_score=False, 
n_jobs=None, 
random_state=1,
warm_start=False, 
class_weight=None, 
ccp_alpha=0.0,
max_samples=None)

In [27]:
# %%time
# parameters = {'n_jobs':[4],
#               'criterion': ["gini", "entropy"], # log_loss 
#               'max_depth':[2, 4, 6, 8, 9, 10, 11, 12, 14, 16, 18, 20],
#               'min_samples_split':[2, 3, 4, 5],
#               'max_features': ["sqrt", "log2", None],
#               'class_weight':['balanced', 'balanced_subsample', None],
#               'ccp_alpha':[0.0, 1, 2, 3], 
#              # 'min_impurity_decreasefloat':[0, 1, 2], # roblem with sqrt stuff?
#               'max_samples':[None, 1, 2, 3, 4, 5]
#              } # , 
# forest_grid_1 = GridSearchCV(RandomForestClassifier(random_state=0), 
#                              parameters, cv=5, verbose=1,
#                              error_score='raise')

# forest_grid_1.fit(x_train_df.iloc[:, 1:], y_train_df.Vote.values.ravel())

# print (forest_grid_1.best_params_)
# print (forest_grid_1.best_score_)

# import pickle

# model_dir = "C:\\Users\\kirti\\Documents\\NASA_data\\ML_Models\\"
# filename = model_dir + 'forest_grid_1.sav'
# pickle.dump(forest_1_default, open(filename, 'wb'))

Fitting 5 folds for each of 20736 candidates, totalling 103680 fits
{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 14, 'max_features': 'log2', 'max_samples': None, 'min_samples_split': 4, 'n_jobs': 4}
0.9562051727885242
CPU times: total: 3h 57min 46s
Wall time: 4h 39min 34s


In [30]:
(((103680 *8)/30)/60)/60

7.680000000000001

In [27]:
%%time
parameters = {'n_jobs':[4],
              'criterion': ["gini"], # log_loss 
              'max_depth':[10, 11, 12, 13, 14, 15],
              'min_samples_split':[4],
              'max_features': ["log2"],
              'class_weight':[None],
              'ccp_alpha':[0.0], 
             # 'min_impurity_decreasefloat':[0, 1, 2], # roblem with sqrt stuff?
              'max_samples':[None]
             } # , 
forest_grid_1 = GridSearchCV(RandomForestClassifier(random_state=0), 
                             parameters, cv=5, verbose=1,
                             error_score='raise')

forest_grid_1.fit(x_train_df.iloc[:, 1:], y_train_df.Vote.values.ravel())

print (forest_grid_1.best_params_)
print (forest_grid_1.best_score_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 14, 'max_features': 'log2', 'max_samples': None, 'min_samples_split': 4, 'n_jobs': 4}
0.9562051727885242
CPU times: total: 4.45 s
Wall time: 8.62 s


In [36]:
import pickle
# model_dir = "/Users/hn/Documents/01_research_data/NASA/ML_Models/"
model_dir = "C:\\Users\\kirti\\Documents\\NASA_data\\ML_Models\\"
filename = model_dir + 'forest_grid_1.sav'
pickle.dump(forest_1_default, open(filename, 'wb'))

In [34]:
forest_grid_1_predictions = forest_grid_1.predict(x_test_df.iloc[:, 1:])
forest_grid_1_y_test_df=y_test_df.copy()
forest_grid_1_y_test_df["prediction"]=list(forest_grid_1_predictions)
forest_grid_1_y_test_df.head(2)

,ID,Vote,prediction
1221,7667_WSDA_SF_2016,1,1
1334,99748_WSDA_SF_2017,1,1


In [41]:
true_single_predicted_single=0
true_single_predicted_double=0

true_double_predicted_single=0
true_double_predicted_double=0

for index_ in forest_grid_1_y_test_df.index:
    curr_vote=list(forest_grid_1_y_test_df[forest_grid_1_y_test_df.index==index_].Vote)[0]
    curr_predict=list(forest_grid_1_y_test_df[forest_grid_1_y_test_df.index==index_].prediction)[0]
    if curr_vote==curr_predict:
        if curr_vote==1: 
            true_single_predicted_single+=1
        else:
            true_double_predicted_double+=1
    else:
        if curr_vote==1:
            true_single_predicted_double+=1
        else:
            true_double_predicted_single+=1
            
forest_grid_1_confus_tbl_test = pd.DataFrame(columns=['None', 'Predict_Single', 'Predict_Double'], 
                               index=range(2))
forest_grid_1_confus_tbl_test.loc[0, 'None'] = 'Actual_Single'
forest_grid_1_confus_tbl_test.loc[1, 'None'] = 'Actual_Double'
forest_grid_1_confus_tbl_test['Predict_Single']=0
forest_grid_1_confus_tbl_test['Predict_Double']=0

forest_grid_1_confus_tbl_test.loc[0, "Predict_Single"]=true_single_predicted_single
forest_grid_1_confus_tbl_test.loc[0, "Predict_Double"]=true_single_predicted_double
forest_grid_1_confus_tbl_test.loc[1, "Predict_Single"]=true_double_predicted_single
forest_grid_1_confus_tbl_test.loc[1, "Predict_Double"]=true_double_predicted_double
forest_grid_1_confus_tbl_test

,None,Predict_Single,Predict_Double
0,Actual_Single,218,1
1,Actual_Double,13,37


In [74]:
FG1_y_test_df_act_1_pred_2=forest_grid_1_y_test_df[forest_grid_1_y_test_df.Vote==1].copy()
FG1_y_test_df_act_2_pred_1=forest_grid_1_y_test_df[forest_grid_1_y_test_df.Vote==2].copy()

FG1_y_test_df_act_1_pred_2=FG1_y_test_df_act_1_pred_2[FG1_y_test_df_act_1_pred_2.prediction==2].copy()
FG1_y_test_df_act_2_pred_1=FG1_y_test_df_act_2_pred_1[FG1_y_test_df_act_2_pred_1.prediction==1].copy()

FG1_y_test_df_act_2_pred_1 = pd.merge(FG1_y_test_df_act_2_pred_1, ground_truth_labels_extended, on=['ID'], how='left')
FG1_y_test_df_act_1_pred_2 = pd.merge(FG1_y_test_df_act_1_pred_2, ground_truth_labels_extended, on=['ID'], how='left')

print (FG1_y_test_df_act_2_pred_1.ExctAcr.sum())
print (FG1_y_test_df_act_1_pred_2.ExctAcr.sum())
print (FG1_y_test_df_act_2_pred_1.ExctAcr.sum()-FG1_y_test_df_act_1_pred_2.ExctAcr.sum())

1054.5154381093193
17.8815032814299
1036.6339348278893


# SG

In [47]:
VI_idx = "EVI"
data_dir = "C:\\Users\\kirti\\Documents\\NASA_data\\VI_TS\\05_SG_TS\\"

file_names = ["SG_Walla2015_" + VI_idx + "_JFD.csv", "SG_AdamBenton2016_" + VI_idx + "_JFD.csv", 
              "SG_Grant2017_" + VI_idx + "_JFD.csv", "SG_FranklinYakima2018_"+ VI_idx +"_JFD.csv"]

data=pd.DataFrame()

for file in file_names:
    curr_file=pd.read_csv(data_dir + file)
    curr_file['human_system_start_time'] = pd.to_datetime(curr_file['human_system_start_time'])
    
    # These data are for 3 years. The middle one is the correct one
    all_years = sorted(curr_file.human_system_start_time.dt.year.unique())
    if len(all_years)==3 or len(all_years)==2:
        proper_year = all_years[1]
    elif len(all_years)==1:
        proper_year = all_years[0]

    curr_file = curr_file[curr_file.human_system_start_time.dt.year==proper_year]
    data=pd.concat([data, curr_file])

data.reset_index(drop=True, inplace=True)
data.head(2)

,ID,human_system_start_time,EVI
0,135073_WSDA_SF_2015,2015-01-10,0.054429
1,135073_WSDA_SF_2015,2015-01-20,0.051731


In [49]:
ground_truth = data[data.ID.isin(list(ground_truth_labels.ID.unique()))].copy()

print (len(meta_moreThan10Acr.ID.unique()))
ground_truth_labels_extended = pd.merge(ground_truth_labels, meta, on=['ID'], how='left')
ground_truth_labels = ground_truth_labels_extended[ground_truth_labels_extended.ExctAcr>=10].copy()
ground_truth_labels.reset_index(drop=True, inplace=True)

print ("There are [{:.0f}] fields in total whose area adds up to [{:.2f}].".format(len(ground_truth_labels_extended), \
                                                                     ground_truth_labels_extended.ExctAcr.sum()))


print ("There are [{:.0f}] fields larger than 10 acres whose area adds up to [{:.2f}].".format(len(ground_truth_labels), \
                                                                    ground_truth_labels.ExctAcr.sum()))


3539
There are [1342] fields in total whose area adds up to [83430.79].
There are [1342] fields larger than 10 acres whose area adds up to [83430.79].


In [50]:
ground_truth = ground_truth[ground_truth.ID.isin((list(meta_moreThan10Acr.ID)))].copy()
ground_truth_labels = ground_truth_labels[ground_truth_labels.ID.isin((list(meta_moreThan10Acr.ID)))].copy()

ground_truth.reset_index(drop=True, inplace=True)
ground_truth_labels.reset_index(drop=True, inplace=True)

In [51]:
ground_truth.sort_values(by=["ID", 'human_system_start_time'], inplace=True)
ground_truth_labels.sort_values(by=["ID"], inplace=True)

ground_truth.reset_index(drop=True, inplace=True)
ground_truth_labels.reset_index(drop=True, inplace=True)

assert (len(ground_truth.ID.unique()) == len(ground_truth_labels.ID.unique()))

print (list(ground_truth.ID)[0])
print (list(ground_truth_labels.ID)[0])
print ("____________________________________")
print (list(ground_truth.ID)[-1])
print (list(ground_truth_labels.ID)[-1])
print ("____________________________________")
print (list(ground_truth.ID.unique())==list(ground_truth_labels.ID.unique()))

100048_WSDA_SF_2017
100048_WSDA_SF_2017
____________________________________
99909_WSDA_SF_2017
99909_WSDA_SF_2017
____________________________________
True


# Widen

In [53]:
EVI_colnames = [VI_idx + "_" + str(ii) for ii in range(1, 37) ]
columnNames = ["ID"] + EVI_colnames
ground_truth_wide = pd.DataFrame(columns=columnNames, 
                                index=range(len(ground_truth.ID.unique())))
ground_truth_wide["ID"] = ground_truth.ID.unique()

for an_ID in ground_truth.ID.unique():
    curr_df = ground_truth[ground_truth.ID==an_ID]
    
    ground_truth_wide_indx = ground_truth_wide[ground_truth_wide.ID==an_ID].index
    ground_truth_wide.loc[ground_truth_wide_indx, "EVI_1":"EVI_36"] = curr_df.EVI.values[:36]

In [54]:
ground_truth_labels = ground_truth_labels.set_index('ID')
ground_truth_labels = ground_truth_labels.reindex(index=ground_truth_wide['ID'])
ground_truth_labels = ground_truth_labels.reset_index()

In [55]:
ground_truth_labels=ground_truth_labels[["ID", "Vote"]]
ground_truth_labels.head(2)

,ID,Vote
0,100048_WSDA_SF_2017,1
1,100081_WSDA_SF_2017,1


In [56]:
x_train_df, x_test_df, y_train_df, y_test_df = train_test_split(ground_truth_wide, 
                                                                ground_truth_labels, 
                                                                test_size=0.2, 
                                                                random_state=0,
                                                                shuffle=True,
                                                                stratify=ground_truth_labels.Vote.values)

In [57]:
%%time
forest_1_default_SG = RandomForestClassifier(n_estimators=100, 
                                          criterion='gini', max_depth=None, 
                                          min_samples_split=2, min_samples_leaf=1, 
                                          min_weight_fraction_leaf=0.0,
                                          max_features='sqrt', max_leaf_nodes=None, 
                                          min_impurity_decrease=0.0, 
                                          bootstrap=True, oob_score=False, n_jobs=None, 
                                          random_state=1, verbose=0, 
                                          warm_start=False, class_weight=None, 
                                          ccp_alpha=0.0, max_samples=None)

forest_1_default_SG.fit(x_train_df.iloc[:, 1:], y_train_df.iloc[:, 1:].values.ravel())

CPU times: total: 375 ms
Wall time: 388 ms


RandomForestClassifier(max_features='sqrt', random_state=1)

In [58]:
forest_1_default_SG_predictions = forest_1_default_SG.predict(x_test_df.iloc[:, 1:])
forest_1_default_SG_y_test_df=y_test_df.copy()
forest_1_default_SG_y_test_df["prediction"]=list(forest_1_default_SG_predictions)
forest_1_default_SG_y_test_df.head(2)

,ID,Vote,prediction
1221,7667_WSDA_SF_2016,1,1
1334,99748_WSDA_SF_2017,1,1


In [59]:
true_single_predicted_single=0
true_single_predicted_double=0

true_double_predicted_single=0
true_double_predicted_double=0

for index_ in forest_1_default_SG_y_test_df.index:
    curr_vote=list(forest_1_default_SG_y_test_df[forest_1_default_SG_y_test_df.index==index_].Vote)[0]
    curr_predict=list(forest_1_default_SG_y_test_df[forest_1_default_SG_y_test_df.index==index_].prediction)[0]
    if curr_vote==curr_predict:
        if curr_vote==1: 
            true_single_predicted_single+=1
        else:
            true_double_predicted_double+=1
    else:
        if curr_vote==1:
            true_single_predicted_double+=1
        else:
            true_double_predicted_single+=1
            
forest_default_SG_confus_tbl_test = pd.DataFrame(columns=['None', 'Predict_Single', 'Predict_Double'], 
                               index=range(2))
forest_default_SG_confus_tbl_test.loc[0, 'None'] = 'Actual_Single'
forest_default_SG_confus_tbl_test.loc[1, 'None'] = 'Actual_Double'
forest_default_SG_confus_tbl_test['Predict_Single']=0
forest_default_SG_confus_tbl_test['Predict_Double']=0

forest_default_SG_confus_tbl_test.loc[0, "Predict_Single"]=true_single_predicted_single
forest_default_SG_confus_tbl_test.loc[0, "Predict_Double"]=true_single_predicted_double
forest_default_SG_confus_tbl_test.loc[1, "Predict_Single"]=true_double_predicted_single
forest_default_SG_confus_tbl_test.loc[1, "Predict_Double"]=true_double_predicted_double
forest_default_SG_confus_tbl_test

,None,Predict_Single,Predict_Double
0,Actual_Single,218,1
1,Actual_Double,13,37


In [70]:
FD1_yTest_df_act_1_pred_2=forest_1_default_SG_y_test_df[forest_1_default_SG_y_test_df.Vote==1].copy()
FD1_yTest_df_act_2_pred_1=forest_1_default_SG_y_test_df[forest_1_default_SG_y_test_df.Vote==2].copy()

FD1_yTest_df_act_1_pred_2=FD1_yTest_df_act_1_pred_2[FD1_yTest_df_act_1_pred_2.prediction==2].copy()
FD1_yTest_df_act_2_pred_1=FD1_yTest_df_act_2_pred_1[FD1_yTest_df_act_2_pred_1.prediction==1].copy()

FD1_yTest_df_act_2_pred_1 = pd.merge(FD1_yTest_df_act_2_pred_1, ground_truth_labels_extended, on=['ID'], how='left')
FD1_yTest_df_act_1_pred_2 = pd.merge(FD1_yTest_df_act_1_pred_2, ground_truth_labels_extended, on=['ID'], how='left')

aa=FD1_yTest_df_act_2_pred_1.ExctAcr.sum()
bb=FD1_yTest_df_act_1_pred_2.ExctAcr.sum()
print ("FD1_yTest_df_act_2_pred_1.ExctAcr.sum(): ",aa)
print ("FD1_yTest_df_act_1_pred_2.ExctAcr.sum(): ", bb)
print (aa-bb)

FD1_yTest_df_act_2_pred_1.ExctAcr.sum():  1101.8806875993102
FD1_yTest_df_act_1_pred_2.ExctAcr.sum():  17.8815032814299
1083.9991843178805


In [60]:
%%time
parameters = {'n_jobs':[4],
              'criterion': ["gini", "entropy"], # log_loss 
              'max_depth':[2, 4, 6, 8, 9, 10, 11, 12, 14, 16, 18, 20],
              'min_samples_split':[2, 3, 4, 5],
              'max_features': ["sqrt", "log2", None],
              'class_weight':['balanced', 'balanced_subsample', None],
              'ccp_alpha':[0.0, 1, 2, 3], 
             # 'min_impurity_decreasefloat':[0, 1, 2], # roblem with sqrt stuff?
              'max_samples':[None, 1, 2, 3, 4, 5]
             } # , 
forest_grid_1_SG = GridSearchCV(RandomForestClassifier(random_state=0), 
                             parameters, cv=5, verbose=1,
                             error_score='raise')

forest_grid_1_SG.fit(x_train_df.iloc[:, 1:], y_train_df.Vote.values.ravel())

print (forest_grid_1_SG.best_params_)
print (forest_grid_1_SG.best_score_)

import pickle

model_dir = "C:\\Users\\kirti\\Documents\\NASA_data\\ML_Models\\"
filename = model_dir + 'forest_grid_1_SG.sav'
pickle.dump(forest_grid_1_SG, open(filename, 'wb'))

Fitting 5 folds for each of 20736 candidates, totalling 103680 fits
{'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 8, 'max_features': 'log2', 'max_samples': None, 'min_samples_split': 3, 'n_jobs': 4}
0.9608693762225604
CPU times: total: 3h 48min 31s
Wall time: 4h 23min 4s


In [61]:
forest_grid_1_SG_predictions = forest_grid_1_SG.predict(x_test_df.iloc[:, 1:])
forest_grid_1_SG_y_test_df=y_test_df.copy()
forest_grid_1_SG_y_test_df["prediction"]=list(forest_grid_1_SG_predictions)
forest_grid_1_SG_y_test_df.head(2)

,ID,Vote,prediction
1221,7667_WSDA_SF_2016,1,1
1334,99748_WSDA_SF_2017,1,1


In [63]:
true_single_predicted_single=0
true_single_predicted_double=0

true_double_predicted_single=0
true_double_predicted_double=0

for index_ in forest_grid_1_SG_y_test_df.index:
    curr_vote=list(forest_grid_1_SG_y_test_df[forest_grid_1_SG_y_test_df.index==index_].Vote)[0]
    curr_predict=list(forest_grid_1_SG_y_test_df[forest_grid_1_SG_y_test_df.index==index_].prediction)[0]
    if curr_vote==curr_predict:
        if curr_vote==1: 
            true_single_predicted_single+=1
        else:
            true_double_predicted_double+=1
    else:
        if curr_vote==1:
            true_single_predicted_double+=1
        else:
            true_double_predicted_single+=1
            
forest_default_SG_confus_tbl_test = pd.DataFrame(columns=['None', 'Predict_Single', 'Predict_Double'], 
                               index=range(2))
forest_default_SG_confus_tbl_test.loc[0, 'None'] = 'Actual_Single'
forest_default_SG_confus_tbl_test.loc[1, 'None'] = 'Actual_Double'
forest_default_SG_confus_tbl_test['Predict_Single']=0
forest_default_SG_confus_tbl_test['Predict_Double']=0

forest_default_SG_confus_tbl_test.loc[0, "Predict_Single"]=true_single_predicted_single
forest_default_SG_confus_tbl_test.loc[0, "Predict_Double"]=true_single_predicted_double
forest_default_SG_confus_tbl_test.loc[1, "Predict_Single"]=true_double_predicted_single
forest_default_SG_confus_tbl_test.loc[1, "Predict_Double"]=true_double_predicted_double
forest_default_SG_confus_tbl_test

,None,Predict_Single,Predict_Double
0,Actual_Single,212,7
1,Actual_Double,12,38


In [72]:
FG1_yTest_df_act_1_pred_2=forest_grid_1_SG_y_test_df[forest_grid_1_SG_y_test_df.Vote==1].copy()
FG1_yTest_df_act_2_pred_1=forest_grid_1_SG_y_test_df[forest_grid_1_SG_y_test_df.Vote==2].copy()

FG1_yTest_df_act_1_pred_2=FG1_yTest_df_act_1_pred_2[FG1_yTest_df_act_1_pred_2.prediction==2].copy()
FG1_yTest_df_act_2_pred_1=FG1_yTest_df_act_2_pred_1[FG1_yTest_df_act_2_pred_1.prediction==1].copy()

FG1_yTest_df_act_2_pred_1 = pd.merge(FG1_yTest_df_act_2_pred_1, ground_truth_labels_extended, on=['ID'], how='left')
FG1_yTest_df_act_1_pred_2 = pd.merge(FG1_yTest_df_act_1_pred_2, ground_truth_labels_extended, on=['ID'], how='left')

print (FG1_yTest_df_act_2_pred_1.ExctAcr.sum())
print (FG1_yTest_df_act_1_pred_2.ExctAcr.sum())
print (np.abs(FG1_yTest_df_act_1_pred_2.ExctAcr.sum() - FG1_yTest_df_act_2_pred_1.ExctAcr.sum()))

1090.27640030951
295.9205731086862
794.3558272008239
